# A Cando Demo
We are going to explore two catalyst designs to look for macromolecules that could wrap around a Zr-O metal cluster and leave open coordination sites that could bind and hydrolyze reactive phosphate esters.

### NOTE: Some/all of the interactive display windows are clipped and display a scroll-bar.  
#### This isn't good for interactive viewing.  
#### Solution: Right click on the window and select "Disable Scrolling For Outputs".

## To replay the calculations in this notebook:

Click on this text box and then hit Shift-Enter to evaluate each cell.

Some cells take a few seconds to minutes to evaluate - wait until output appears before moving on to the next cell.

### We will start with a chemdraw structure for a designed molecule that presents six carboxylic acids that could displace benzoic acids from a crystal structure of a Zr-O cluster.

It turns out that this is a poor design - but suggests some ideas on how to make a better one.

Chemdraw was used to draw the structure - we will use the saved chemdraw file to build a three-dimensional structure of the catalyst design and wrap it around a Zr-O cluster to see if the resulting geometry is chemically reasonable.

In [ ]:
(jupyter:png-file "data/cat-design/trimer.png")

### First we will load the X-ray crystal structure of the Zr-O cluster.

It is surrounded by benzoic acid groups (I don't have the Cambridge Crystallographic Database accession number in hand when I created this demo).

In [ ]:
(defparameter *zr* (load-mol2 "data/cat-design/renamed-zrcluster.mol2"))

#### You can look at the structure and manipulate it using your mouse.

Rotate the molecule by dragging the left mouse button (or trackpad) within the view below.

Zoom (two button drag scroll or scroll wheel).

Pick atoms (hover over an atom and left button click).

At the bottom right corner of the view above is a very small control, drag that to make the view larger or smaller.

Double click with left button to go full screen, hit Escape to return to browser.

In [ ]:
(defparameter *zr-view* (show *zr*)) *zr-view*

## Load and build 3D coordinates of trimer

In [ ]:
(defparameter *agg* (load-chem-draw-aggregate "data/cat-design/trimer.cdxml"))

In [ ]:
(chem:fill-in-implicit-hydrogens *agg*)
(build-unbuilt-hydrogens *agg*)

In [ ]:
(show *agg*)

The structure above is the Chemdraw structure with no depth.  It illustrates that the Chemdraw structure is loaded successfuly into Cando.

In [ ]:
(defparameter *stereocenters* (stereocenters-sorted-by-name *agg*))

In [ ]:
(set-all-stereocenters-to *stereocenters* :S :show t)

We randomly displace all atoms to assist the energy minimization that we are about to do below.

In [ ]:
(jostle *agg* 20)

In [ ]:
(show *agg*)

In the view above, atoms are randomly jostled in space to break them out of "flatland".

## Below we construct a chemically reasonable structure of the molecule by minimizing the conformation energy. 

This does a lot of things in sequence:

1) Cando automatically assigns atom types.

2) Cando assigns Generalized Amber Force Field (GAFF) parameters.

3) Cando minimizes the energy of the structure, automatically switching between three energy minimizers to take advantage of their different speed/convergence characteristics.

In [ ]:
(easy-gaff)

#### You can ignore the warnings above.

In [ ]:
(build-unbuilt-hydrogens *agg*)

In [ ]:
(energy:minimize *agg* 
                 :restraints-on t
                 :max-sd-steps 5000
                 :sd-tolerance 5000.0
                 :max-cg-steps 1000)

In [ ]:
(show *agg*)

## Above should be a chemically reasonable three-dimensional structure of the molecule.

In [ ]:
(cando-jupyter:center-on *agg* '(0.0 0.0 15.0))

In [ ]:
(defparameter *design-view* (show *zr*))
(defparameter *cagg* (make-instance 'cando-structure :matter *agg*))
(nglv::add-structure *design-view* *cagg*)
*design-view*

## You can select pairs of carboxylic acid carbons on the ligand and on the benzoic acids of the Zr-O cluster multiple times to build a mapping of atoms on the ligand to the cluster.  It's a bit tricky and takes a couple of tries to get it right.  The function below pulls out the mapping.

In [ ]:
(defparameter *named-map* '((:C_350 . :C_2) (:C_354 . :C_1) 
                  (:C_255 . :C_7) (:C_259 . :C_8)
                  (:C_185 . :C_3) (:C_196 . :C_4)))

In [ ]:
(cando-jupyter:cartoon-atom-map *design-view* *named-map* *agg* *zr*)

## Scroll up and you should see magenta lines joining the pairs of atoms that are mapped on to each other.

### Next we are going to constrain the selected atoms from the trial catalyst to the Zr-O crystal structure.

In [ ]:
(clear-all-atom-restraints *agg*)

In [ ]:
(cando-jupyter:anchor-named-atom-map *named-map* *agg* *zr*)

In [ ]:
(set-all-stereocenters-to *stereocenters* :S :show t)

In [ ]:
(energy:minimize *agg* 
                 :restraints-on t
                 :max-sd-steps 1000
                 :sd-tolerance 5000.0
                 :max-cg-steps 1000)

In [ ]:
(show *agg*)

### It turns out that the structure above is a bad design - with lots of bad geometry.
# We can rule this design out and move on to another design with a larger macrocycle to better envelop the Zr-O cluster.

### Overlay of the trial above on the Zr-O cluster and looking at it closely can help us learn what is wrong with the design.

In [ ]:
(defparameter *overlay-view* (show *zr*))
(defparameter *cagg* (make-instance 'cando-structure :matter *agg*))
(nglv::clear-representations *overlay-view*)
(nglv::add-representation *overlay-view* "hyperball")
(nglv::add-structure *overlay-view* *cagg*)
*overlay-view*

In [ ]:
(jupyter:png-file "data/cat-design/trial-catalyst2.png")

In [ ]:
(defparameter *agg2* (load-chem-draw-aggregate "data/cat-design/trial-catalyst2.cdxml"))

In [ ]:
(chem:fill-in-implicit-hydrogens *agg2*)
(build-unbuilt-hydrogens *agg2*)

In [ ]:
(show *agg2*)

### The structure above is the Chemdraw structure with no depth.  It illustrates that the Chemdraw structure is loaded successfuly into Cando.

In [ ]:
(defparameter *stereocenters2* (stereocenters-sorted-by-name *agg2*))

In [ ]:
(set-all-stereocenters-to *stereocenters2* :S :show t)

In [ ]:
(jostle *agg2* 20)

In [ ]:
(show *agg2*)

In [ ]:
(energy:minimize *agg2* 
                 :restraints-on t
                 :max-sd-steps 2000
                 :sd-tolerance 5000.0
                 :max-cg-steps 1000)

In [ ]:
(show *agg2*)

In [ ]:
(cando-jupyter:center-on *agg2* '(0.0 0.0 15.0))

In [ ]:
(defparameter *design-view2* (show *zr*))
(defparameter *cagg2* (make-instance 'cando-structure :matter *agg2*))
(nglv::add-structure *design-view2* *cagg2*)
*design-view2*

In [ ]:
(defparameter *named-map2* '((:C_452 . :C_8) (:C_458 . :C_7) 
                    (:C_198 . :C_5) (:C_219 . :C_6)
                    (:C_305 . :C_4) (:C_311 . :C_3)))

In [ ]:
(cando-jupyter:cartoon-atom-map *design-view2* *named-map2* *agg2* *zr*)

## Scroll up and you should see magenta lines joining the pairs of atoms that are mapped on to each other.

### Next we are going to constrain the selected atoms from the trial catalyst to the Zr-O crystal structure.

In [ ]:
(clear-all-atom-restraints *agg2*)

In [ ]:
(cando-jupyter:anchor-named-atom-map *named-map2* *agg2* *zr*)

In [ ]:
(set-all-stereocenters-to *stereocenters2* :S :show t)

In [ ]:
(energy:minimize *agg2* 
                 :restraints-on t
                 :max-sd-steps 2000
                 :sd-tolerance 5000.0
                 :max-cg-steps 1000)

In [ ]:
(show *agg2*)

### This may be a better design.
### It's going to take more work to establish this.
### A build-tree will be constructed and ligands will be restrained to mimic the benzoic acids from the Zr-O cluster.
### 1) Rotatable bonds will be twisted.
### 2) Stereocenters will be flipped.
### 3) Functional groups will be varied.
### While scoring the conformational energy to find an optimal design.

## That's it for now - anything more sophisticated requires setting up more designs and running more sophisticated calculations on super-computers.
## This user interface can drive 100,000s of copies of Cando running on large super-computers.